In [17]:
import os, time, datetime, hashlib, json, base64
import numpy as np
from bridge import Bridge
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from PIL import Image
from eolearn.core import EOWorkflow, FeatureType, OutputTask, linearly_connect_tasks
from eolearn.io import SentinelHubInputTask
from sentinelhub import CRS, BBox, DataCollection, SHConfig
import boto3
from botocore.exceptions import ClientError
import cv2

config = ""
cwd = os.getcwd()
export_folder = os.path.join(cwd, "data")
num_of_imgs = 0

aoi_bbox = BBox(bbox=[5.60, 52.68, 5.75, 52.63], crs=CRS.WGS84)
time_interval = ("2018-04-01", "2018-05-01")
maxcc = 0.5
resolution = 10
time_difference = datetime.timedelta(hours=2)

def create_workflow() -> EOWorkflow:
    # Get Sentinel-2 data
    l1c_task1 = SentinelHubInputTask(
            data_collection=DataCollection.SENTINEL2_L1C,
            bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"],
            bands_feature=(FeatureType.DATA, "L1C_data"),
            additional_data=[(FeatureType.MASK, "dataMask")],
            resolution= resolution,
            maxcc=maxcc,
            time_difference = time_difference,
            config = config,
            max_threads=3,
        )
    # SAVE EOPATCH IN MEMORY
    output_task = OutputTask("eopatch")
    workflow_nodes = linearly_connect_tasks(l1c_task1, output_task)
    workflow = EOWorkflow(workflow_nodes)
    return workflow
            
def request(workflow):
    h = ""
    url = ""
    workflow_nodes = workflow.get_nodes()
    # Allows you to pass special parameters into the nodes before execution
    result = workflow.execute({
            workflow_nodes[0]: {"bbox": aoi_bbox, "time_interval": time_interval}
    })
    if result.workflow_failed():
        print("Error: Workflow failed")
        raise Exception("Workflow failed.")
    else:
        eopatch = result.outputs["eopatch"]
        data = eopatch.__getitem__((FeatureType.DATA, "L1C_data"))
        h, url = get_payload(data)
    
    return h, url
    
"""
Configures SentinelHub OAuth access and returns config object
"""
def sh_signin():
    config = SHConfig()
    config.instance_id = '81f4ec4b-3952-4c9d-91bd-2e0536d86e9a'
    config.sh_client_id = '9b276c05-d06d-449e-8369-1b521fd97174'
    config.sh_client_secret = 'wX7cF<&K,euia!?nuDx!>F7,j,b{8k;5w/&edd1x'
    config.save()
    return config

"""
Function that takes in the result of an eopath which is an ndarray of shape:
    (time * height * width * band) and saves only the RGB bands as images.
"""
def save_rgb_bands(data):
    band_name = ""
    for i in range(data.shape[0]):
        imgs = data[i, ...]
        for j in range(imgs.shape[-1]):
            if(j >= 2 & j <= 4):
                if(j == 2):
                    band_name= "blue"
                elif(j == 3):
                    band_name="green"
                else:
                    band_name="red"
                band = imgs[..., j]
                scaled_band = (band * 255 / np.max(band)).astype('uint8')
                im = Image.fromarray(scaled_band)
                folder_path = os.path.join(os.getcwd(), "data", "test1rgb", f"img{i}")
                file_path = os.path.join(folder_path, f"img{i}{band_name}.png")
                os.makedirs(folder_path, exist_ok=True)
                im.save(file_path)


def save_img(folder_path: str, img: Image) -> None:
    file_name = get_image_hash(img)
    file_path = os.path.join(folder_path, f"{file_name}.png")
    os.makedirs(folder_path, exist_ok=True)
    img.save(file_path)

def save_img(folder_path: str, img: Image, file_name: str) -> None:
    file_path = os.path.join(folder_path, f"{file_name}.png")
    os.makedirs(folder_path, exist_ok=True)
    img.save(file_path) 

"""
Function that takes in the result of an eopath which is an ndarray of shape:
    (time * height * width * band) and saves ALL bands as images
"""
def save_bands(data):
    for i in range(data.shape[0]):
        imgs = data[i, ...]
        for j in range(imgs.shape[-1]):
            band = imgs[..., j]
            scaled_band = (band * 255 / np.max(band)).astype('uint8')
            im = Image.fromarray(scaled_band)
            folder_path = os.path.join(os.getcwd(), "data", "test1rgb", f"img{i}")
            file_path = os.path.join(folder_path, f"img{i}band{j}.png")
            os.makedirs(folder_path, exist_ok=True)
            im.save(file_path)

def get_image_sets(data, start, end):
    img_sets = []
    for i in range(data.shape[0]):
        if(i >= start & i <= end):
            img_sets.append(data[i, ...])
    return img_sets


# Returns list of images in SentinelHub representation: x * y
def get_image_sets(data):
    img_sets = []
    for i in range(data.shape[0]):
        img_sets.append(data[i, ...])
    return img_sets
    
def get_image_set(data, index) -> np.ndarray:
    for i in range(data.shape[0]):
        if (i == index):
            img = data[i, ...]
            return img

def get_bands(img):
    band_imgs = []
    for i in range(img.shape[-1]):
        band = img[..., i]
        scaled_band = (band * 255 / np.max(band)).astype('uint8')
        band_img = Image.fromarray(scaled_band)
        band_imgs.append(band_img)
    return band_imgs

def get_band(img, index) -> Image:
    for i in range(img.shape[-1]):
        if (i == index):
            band = img[..., i]
            scaled_band = (band * 255 / np.max(band)).astype('uint8')
            band_img = Image.fromarray(scaled_band)
            return band_img

# Recieves image in SentinelHub representation and returns

def get_band_cv2array(img_set, index) -> np.ndarray:
    for i in range(img_set.shape[-1]):
        if (i == index):
            band_img = ""
            band = img_set[..., i]
            scaled_band = (band * 255 / np.max(band)).astype('uint8')
           
            band_img = Image.fromarray(scaled_band)
        
            rgb_band_img = band_img.convert('RGB')
            rgb_band_img_arr = np.asarray(rgb_band_img)
            
            return rgb_band_img_arr 

def get_base64_hash(base64_msg: str) -> str:
    base64_str = base64_msg.encode('utf-8')
    t = str(time.time()).encode('utf-8')

    hasher = hashlib.sha1()
    hasher.update(base64_str)
    hasher.update(t)
    return hasher.hexdigest()

def get_image_hash(img: Image) -> str:
    img_str = image_to_base64string(img).encode('utf-8')
    t = str(time.time()).encode('utf-8')

    hasher = hashlib.sha1()
    hasher.update(img_str)
    hasher.update(t)
    return hasher.hexdigest()

"""
Takes in image path and returns base64 string
"""
def imagepath_to_base64string(image_path: str):
    with open(image_path, 'rb') as binary_file:
        binary_file_data = binary_file.read()
        base64_bytes = base64.b64encode(binary_file_data)
        base64_message = base64_bytes.decode('utf-8')
    return base64_message

"""
Takes in image and returns base64 string
"""
def image_to_base64string(img: Image):
    img_bytes = img.tobytes()
    base64_bytes = base64.b64encode(img_bytes)
    base64_message = base64_bytes.decode('utf-8')
    return base64_message

"""
Takes in base64 string and returns image in bytes
"""
def base64StringToImage(base64_message):
    #str to ascii bytes
    base64_bytes = base64_message.encode('ascii')
    #ascii bytes to image bytes
    img_bytes = base64.b64decode(base64_bytes) 
    return img_bytes

def smallest_not_zero(arr):
    #print("smallest_not_zero")
    second_min = max(arr)
    idx = 0
    for i in range(len(arr)):
        val = arr[i]
        if((val != 0) and (val < second_min)):
            second_min = val
            idx = i
    return second_min, idx

def calc_mse(imgs):
    mse_results = []
    for i in range(num_of_imgs):
        img_results = []
        img = imgs[i]

        for j in range(num_of_imgs):
            if (i != j):
                ms_err = mean_squared_error(img, imgs[j])
                img_results.append(ms_err)
            else: 
                img_results.append(0)
        mse_results.append(img_results)
    #print("made it out of calc_mse")
    return mse_results
"""
input:
- imgs: list of ndarrays representing images
"""
def calc_ssim(imgs):
    ssim_results = []
    bw_imgs = []
    for i in range(num_of_imgs):
        img_cv2array = imgs[i]
        try: 
            img_temp = cv2.cvtColor(img_cv2array, cv2.COLOR_BGR2GRAY)
        except Exception as e:
            print("error in calc_ssim", e)
        bw_imgs.append(img_temp)
    for i in range(num_of_imgs):
        img_results = []
        img = bw_imgs[i]
        for j in range(num_of_imgs):
            if (i != j):
                ssim_res = ssim(img, bw_imgs[j])
                img_results.append(ssim_res)
            else: 
                img_results.append(0)
        ssim_results.append(img_results)
    return ssim_results

"""
CALC_AVG_SCENARIO that takes in list of MSE scores and SSIM scores per image where the index in the list
corresponds to an image with the same index in the image list passed to compute mse_results and 
ssim_results.

    input:
    - mse_results: List containing MSE values for each image pair combination.
    - ssim_results: List containing SSIM values for each image pair combination.
    output:
    - idx: index of image that is the average scenario based on MSE and SSIM results.
"""
def calc_avg_scenario(mse_results, ssim_results):
    #print("num of images", num_of_imgs)
    avg_scenario_results = [0] * num_of_imgs
    for i in range(num_of_imgs):
        mse_result = mse_results[i]
        ssim_result = ssim_results[i]

        mse_min, best_sme_idx = smallest_not_zero(mse_result)
        #print("ssim_result")
        ssim_max = max(ssim_result)
        
        best_ssim_idx = ssim_result.index(ssim_max)

        if(best_ssim_idx == best_sme_idx):
            avg_scenario_results[best_sme_idx] += 2
        else:
            avg_scenario_results[best_ssim_idx] += 1
            avg_scenario_results[best_sme_idx] += 1


    print("avg_sc_results")    
    idx = avg_scenario_results.index(max(avg_scenario_results))
    return idx

def get_payload(data):
    # List containing all Band 2 images of images to be compared.
    global num_of_imgs
    # List of images as cv2 ndarray representation.
    imgs = []
    h = ""
    avg_scenario_idx = 0
    avg_scenario = ""
    url = ""

    img_sets = get_image_sets(data)
    for i in range(len(img_sets)):
        img_set = img_sets[i]
        # Gets the blue channel
        imgs.append(get_band_cv2array(img_set, 2))

    num_of_imgs = len(imgs)

    mse_results = calc_mse(imgs)
    ssim_results = calc_ssim(imgs)

    avg_scenario_idx = calc_avg_scenario(mse_results, ssim_results)
    avg_scenario = imgs[avg_scenario_idx]
    avg_scenario = Image.fromarray(avg_scenario)

    h = get_image_hash(avg_scenario)
    url = get_image_url(avg_scenario, h)

    print(h, url)
    return h, url

def get_image_url(im: Image, key: str) -> str:
    url = ""
    bucket_name = "iris-web3athon-demo"
    location = ""

    avg_sc_folder_path = os.path.join(export_folder, "average_scenario")
    avg_sc_name = "avg_sc"
    avg_sc_path = os.path.join(avg_sc_folder_path, avg_sc_name)
    
    save_img(avg_sc_folder_path, im, avg_sc_name)
    
    s3 = boto3.client('s3')
    location = s3.get_bucket_location(Bucket=bucket_name)

    try:
        response = s3.upload_file(avg_sc_path + ".png", bucket_name, key)
    except ClientError as e:
        print("AWS Client Error:", e)
    except Exception as e:
        print("Unspecified error uploading file", e)

    try:
        url = "https://s3-%s.amazonaws.com/%s/%s" % (location, bucket_name, key)
    except Exception as e:
        print("Error getting URL", e)

    return url

In [18]:
config = sh_signin()
bridge = Bridge()
workflow = create_workflow()
try:
    request(workflow)
except Exception as e:
    print("Error", e)

avg_sc_results
6d9357b1b6197a4338606c99f686bff402877a24 https://s3-{'ResponseMetadata': {'RequestId': '7YZJH70EE1RN3Z3Y', 'HostId': 'agoenPPoTQI56aoDgnVTqq73biGO+thkdNDL1ePS/6ayIuaZkmvDD3XCpDjI9bV47tXlCDtNwQI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'agoenPPoTQI56aoDgnVTqq73biGO+thkdNDL1ePS/6ayIuaZkmvDD3XCpDjI9bV47tXlCDtNwQI=', 'x-amz-request-id': '7YZJH70EE1RN3Z3Y', 'date': 'Tue, 20 Sep 2022 04:52:49 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'LocationConstraint': None}.amazonaws.com/iris-web3athon-demo/6d9357b1b6197a4338606c99f686bff402877a24
